In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Explore the data**

In [ ]:
df = pd.read_csv("/kaggle/input/covid-19/data/countries-aggregated.csv")
df

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df["Date"] = pd.to_datetime(df["Date"])
df.info()

# **Preparing the data**

In [ ]:
# Include only data needed for DataFrame
df = df[["Date","Country", "Confirmed", "Deaths"]]
df["Death rate"] = round(df["Deaths"] / df["Confirmed"] * 100, 2)
# Get data for Namibia only
namibia = df["Country"].str.contains("Namibia")
df[namibia]

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.style as style

matplotlib.style.use("bmh")

# **Namibia date rate**

In [ ]:
ax = df[namibia].plot(figsize = (17,8), x = "Date", y = "Death rate")
plt.title("Covid-19 : Namibia death rate")
ax.set_ylabel("Death rate expressed in percentages")

# **World death rate**

In [ ]:
world = df.groupby("Date").sum()
del world["Death rate"]
world["Death rate"] = round(world["Deaths"] / world["Confirmed"] * 100, 2)
world

In [ ]:
ax = world.plot(figsize = (17,8), y = "Death rate")
plt.title("Covid-19 : World death rate")
ax.set_ylabel("Death rate expressed in percentages")

# **World vs Namibia death rate**

In [ ]:
combined_df = world.merge(df[namibia],on= "Date")
combined_df

In [ ]:
combined_df.rename(columns = {"Death rate_x" : "world death rate", "Death rate_y" : "namibia death rate"}, inplace = True)

In [ ]:
ax = combined_df.plot(figsize = (17,8), x = "Date", y = "world death rate")
ax2 = combined_df.plot(figsize = (17,8), x = "Date", y = "namibia death rate", ax = ax)
plt.title("Covid-19 : World vs Namibia death rate")
ax.set_ylabel("Death rate expressed in percentages")

In [ ]:
combined_df['Month'] = pd.to_datetime(df['Date']).dt.to_period('M')
monthly_df = combined_df.groupby("Month").sum()
monthly_df

In [ ]:
monthly_df["world death rate"] = round(monthly_df["Deaths_x"] / monthly_df["Confirmed_x"] * 100, 2)
monthly_df["namibia death rate"] = round(monthly_df["Deaths_y"] / monthly_df["Confirmed_y"] * 100, 2)
monthly_df

In [ ]:
ax3 = monthly_df[["world death rate", "namibia death rate"]].plot(figsize = (17,8), kind = "bar")
plt.title("Covid-19 : World vs Namibia death rate per month")
ax3.set_ylabel("Death rate expressed in percentages")
plt.xticks(rotation = 45)

# **Population density**
<img src="https://upload.wikimedia.org/wikipedia/commons/b/b6/Population_density_of_countries_2018_world_map%2C_people_per_sq_km.svg" width="???px">

In [ ]:
# least dense neighboring countries
angola = df["Country"].str.contains("Angola")
ang = df[angola]
botswana = df["Country"].str.contains("Botswana")
bot = df[botswana]
zambia = df["Country"].str.contains("Zambia")
zam = df[zambia]

nam = df[namibia]
nam

In [ ]:
least_dense_data_frames = [ang, bot, zam]

from functools import reduce

least_dense_df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                            how='inner'), least_dense_data_frames)

least_dense_df_merged.rename(columns = {"Death rate" : "Zambia death rate",
                                "Death rate_y" : "Botswana death rate",
                                "Death rate_x" : "Angola death rate"}, inplace = True)

least_dense_df_merged = least_dense_df_merged.merge(nam, on = "Date")
least_dense_df_merged.rename(columns = {"Death rate" : "Namibia death rate"}, inplace = True)

from_july = least_dense_df_merged["Date"] > "2020-07-01"

least_dense_df_merged[from_july]

# **Plot Namibia vs least dense neighbouring countries**

In [ ]:
ax = least_dense_df_merged[from_july].plot(figsize = (17,8), x = "Date", y = "Namibia death rate", color = "y")
ax2 = least_dense_df_merged[from_july].plot(figsize = (17,8), x = "Date", y = "Angola death rate", ax = ax)
ax2 = least_dense_df_merged[from_july].plot(figsize = (17,8), x = "Date", y = "Botswana death rate", ax = ax)
ax2 = least_dense_df_merged[from_july].plot(figsize = (17,8), x = "Date", y = "Zambia death rate", ax = ax)
plt.title("Covid deaths rates of Namibia and her least dense neighbouring countries")
ax.set_ylabel("Death rate expressed in percentages")
plt.xticks(rotation = 45)

In [ ]:
# more dense neighboring countries
south_africa = df["Country"].str.contains("South Africa")
saf = df[south_africa]
zimbabwe = df["Country"].str.contains("Zimbabwe")
zim = df[zimbabwe]

In [ ]:
more_dense_data_frames = [saf, zim]

from functools import reduce

more_dense_df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                            how='inner'), more_dense_data_frames)

more_dense_df_merged.rename(columns = {"Death rate_y" : "Zimbabwe death rate",
                                        "Death rate_x" : "South Africa death rate"}, inplace = True)

more_dense_df_merged = more_dense_df_merged.merge(nam, on = "Date")
more_dense_df_merged.rename(columns = {"Death rate" : "Namibia death rate"}, inplace = True)

more_dense_df_merged[from_july]

# **Plot Namibia vs more dense neighbouring countries**

In [ ]:
ax = more_dense_df_merged[from_july].plot(figsize = (17,8), x = "Date", y = "Namibia death rate", color = "y")
ax2 = more_dense_df_merged[from_july].plot(figsize = (17,8), x = "Date", y = "South Africa death rate", ax = ax)
ax2 = more_dense_df_merged[from_july].plot(figsize = (17,8), x = "Date", y = "Zimbabwe death rate", ax = ax)
plt.title("Covid deaths rates of Namibia and her more dense neighbouring countries")
ax.set_ylabel("Death rate expressed in percentages")
plt.xticks(rotation = 45)

# **Plot Namibia vs other least dense  countries**
Data used : https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population_density


| Rank | Country | Population density (people per km2)
| --- | --- | --- |
| 194 | Mongolia | 2.07 |
| 193 | Namibia | 2.93 |
| 192 | Australia | 3.00 |
| 191 | Suriname | 3.47 |
| 190 | Iceland | 3.56 |


In [ ]:
mongolia = df["Country"].str.contains("Mongolia")
mong = df[mongolia]
australia = df["Country"].str.contains("Australia")
aus = df[australia]
suriname = df["Country"].str.contains("Suriname")
sur = df[suriname]
iceland = df["Country"].str.contains("Iceland")
ice = df[iceland]

In [ ]:
bottom_5_least_dense_data_frames = [mong, aus, sur]


bottom_5_least_dense_df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                            how='inner'), bottom_5_least_dense_data_frames)

bottom_5_least_dense_df_merged.rename(columns = {"Death rate_y" : "Australia death rate",
                                                 "Death rate_x" : "Mongolia death rate", 
                                                "Death rate" : "Suriname death rate"}, inplace = True)

bottom_5_least_dense_df_merged = bottom_5_least_dense_df_merged.merge(nam, on = "Date")
bottom_5_least_dense_df_merged.rename(columns = {"Death rate" : "Namibia death rate"}, inplace = True)

bottom_5_least_dense_df_merged = bottom_5_least_dense_df_merged.merge(ice, on = "Date")
bottom_5_least_dense_df_merged.rename(columns = {"Death rate" : "Iceland death rate"}, inplace = True)

bottom_5_least_dense_df_merged["World death rate"] = 2.15

In [ ]:
ax = bottom_5_least_dense_df_merged [from_july].plot(figsize = (17,8), x = "Date", y = "Mongolia death rate", color = "y")
ax2 = bottom_5_least_dense_df_merged [from_july].plot(figsize = (17,8), x = "Date", y = "Australia death rate", ax = ax)
ax3 = bottom_5_least_dense_df_merged [from_july].plot(figsize = (17,8), x = "Date", y = "Suriname death rate", ax = ax)
ax4 = bottom_5_least_dense_df_merged [from_july].plot(figsize = (17,8), x = "Date", y = "Namibia death rate", ax = ax)
ax5 = bottom_5_least_dense_df_merged [from_july].plot(figsize = (17,8), x = "Date", y = "Iceland death rate", ax = ax)
ax6 = bottom_5_least_dense_df_merged [from_july].plot(figsize = (17,8), x = "Date", y = "World death rate", ax = ax, color = "black")
plt.title("Covid deaths rates of bottom 5 least densely populated countries")
ax.set_ylabel("Death rate expressed in percentages")
plt.xticks(rotation = 45)

# **Plot Namibia vs most dense countries**
Data used : https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population_density


| Rank | Country | Population density (people per km2)
| --- | --- | --- |
| 1 | Monaco | 18,960.40 |
| 2 | Singapore | 7,894.26 |
| 3 | Bahrain | 1,912.72|
| 4 | Malta | 1,633.54 |
| 5 | Maldives | 1,257.63 |
| 193 | Namibia | 2.93 |


In [ ]:
monaco = df["Country"].str.contains("Monaco")
mon = df[monaco]
singapore = df["Country"].str.contains("Singapore")
sing = df[singapore]
bahrain = df["Country"].str.contains("Bahrain")
bah = df[bahrain]
malta = df["Country"].str.contains("Malta")
malt = df[malta]
maldives = df["Country"].str.contains("Maldives")
mald = df[maldives]

In [ ]:
top_5_dense_data_frames = [mon, sing]


top_5_dense_df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Date'],
                                            how='inner'), top_5_dense_data_frames)

top_5_dense_df_merged.rename(columns = {"Death rate_y" : "Singapore death rate",
                                                 "Death rate_x" : "Monaco death rate"}, inplace = True)


top_5_dense_df_merged = top_5_dense_df_merged.merge(bah, on = "Date")
top_5_dense_df_merged.rename(columns = {"Death rate" : "Bahrain death rate"}, inplace = True)

top_5_dense_df_merged = top_5_dense_df_merged.merge(malt, on = "Date")
top_5_dense_df_merged.rename(columns = {"Death rate" : "Malta death rate"}, inplace = True)

top_5_dense_df_merged = top_5_dense_df_merged.merge(mald, on = "Date")
top_5_dense_df_merged.rename(columns = {"Death rate" : "Maldives death rate"}, inplace = True)

top_5_dense_df_merged = top_5_dense_df_merged.merge(nam, on = "Date")
top_5_dense_df_merged.rename(columns = {"Death rate" : "Namibia death rate"}, inplace = True)

top_5_dense_df_merged["World death rate"] = 2.15
top_5_dense_df_merged

In [ ]:
ax = top_5_dense_df_merged [from_july].plot(figsize = (17,8), x = "Date", y = "Monaco death rate", color = "y")
ax2 = top_5_dense_df_merged [from_july].plot(figsize = (17,8), x = "Date", y = "Singapore death rate", ax = ax)
ax3 = top_5_dense_df_merged [from_july].plot(figsize = (17,8), x = "Date", y = "Bahrain death rate", ax = ax)
ax4 = top_5_dense_df_merged [from_july].plot(figsize = (17,8), x = "Date", y = "Malta death rate", ax = ax)
ax5 = top_5_dense_df_merged [from_july].plot(figsize = (17,8), x = "Date", y = "Maldives death rate", ax = ax)
ax6 = top_5_dense_df_merged [from_july].plot(figsize = (17,8), x = "Date", y = "Namibia death rate", ax = ax)
ax7 = top_5_dense_df_merged [from_july].plot(figsize = (17,8), x = "Date", y = "World death rate", ax = ax, color = "black")
plt.title("Covid deaths rates of top 5 densely populated countries vs Namibia")
ax.set_ylabel("Death rate expressed in percentages")
plt.xticks(rotation = 45)